In [5]:
import pandas as pd
import os

from datetime import datetime

In [6]:
df_fazendas = pd.read_excel('../sample_data/fazendas.xlsx')
df_colheitaderas = pd.read_excel('../sample_data/frentes_de_colheita.xlsx')
map_dict = {0: 'BL.103', 1: 'BL.104', 2: 'BL.110', 3: 'BL.157', 4: 'BL.164', 5: 'BL.173', 6: 'BL.175', 7: 'BL.178', 8: 'BL.189', 9: 'BL.191'}

In [7]:
def __calculate_final_date(df):
    for i in range(len(df)):
        if i == 0:
            df.loc[i, 'data_final'] = df.loc[i, 'data_inicial'] + pd.to_timedelta(df.loc[i, 'dias'], unit='d')
        else:
            df.loc[i, 'data_inicial'] = df.loc[i-1, 'data_final']
            df.loc[i, 'data_final'] = df.loc[i, 'data_inicial'] + pd.to_timedelta(df.loc[i, 'dias'], unit='d')

    return df


def __explode_data_between_dates(df):
    df_dias = pd.DataFrame()

    for i in range(len(df)):
        n_dias = df.loc[i, 'dias']

        for j in range(n_dias):
            temp_df = df.loc[i, :].copy()
            temp_df['dia'] = df.loc[i, 'data_inicial'] + pd.to_timedelta(j, unit='d')

            df_dias = df_dias.append(temp_df)

    return df_dias
    

def exec_process(df_f, df_c, map_dict, path='../outputs/'):
    dir_list = os.listdir(path)
    df_final = pd.DataFrame()

    for i, file in enumerate(dir_list):
        temp_df = pd.read_excel(path+file)
        temp_df['frente_de_colheita'] = map_dict[i]

        temp_df['data_inicial'] = None
        temp_df.loc[0, 'data_inicial'] = datetime.today().date()

        temp_df = temp_df.merge(df_f, how='left', on='cod_fazenda')
        temp_df = temp_df.merge(df_c, how='left', on='frente_de_colheita')

        temp_df['dias'] = (temp_df['producao_(t)'] / temp_df['capacidade_(ton/dia)'] + 1).astype(int)

        temp_df2 = __calculate_final_date(temp_df)

        temp_df3 = __explode_data_between_dates(temp_df2)

        df_final = df_final.append(temp_df3)
    
    df_final = df_final.convert_dtypes()

    return df_final

In [8]:
teste_1 = exec_process(df_fazendas, df_colheitaderas, map_dict)

In [15]:
relatorio_diario = teste_1.groupby('dia', as_index=False).agg({'atr': 'mean', 'dist_(km)': 'mean', 'producao_(t)': 'sum', 'area_(ha)': 'sum', 'frente_de_colheita': 'nunique'})
relatorio_sequenciamento = teste_1.loc[:, ['cod_fazenda', 'frente_de_colheita', 'dia']]

In [18]:
relatorio_diario.to_excel('relatorios/relatorio_diario.xlsx', index=False)
relatorio_sequenciamento.to_excel('relatorios/relatorio_sequenciamento.xlsx', index=False)